In [31]:
import pandas as pd
import numpy as np
from random import randint
from datetime import datetime, timedelta

In [6]:
space_count = 2
seen_name = False
prod_dict = dict()
prod_name, prod_price = None, None
with open('items.txt', 'r') as f:
  count = 1
  for idx, line in enumerate(f.read().split('\n')):
    if space_count == 2:
      prod_name = line.split(':')[0]
      prod_price = line.split(':')[1].strip()
      space_count = 0
      seen_name = True
    elif seen_name:
      seen_name = False
      prod_dict[count] = {
          'prod': [prod_name],
          'price': prod_price,
          'desc': line
      }
      count = count + 1
    else:
      space_count = space_count + 1

In [7]:
prod_dict

{1: {'desc': 'Camera 4500mAH lithium-ion battery (non-removable);Main camera features: Clear, high-resolution photos, Nightscape, Automated AI Photo Enhancer, Portrait mode, and filters.;In-display fingerprint sensor.;Front camera features - Face Unlock, HDR, Screen Flash, face retouching, and filters.;Thickness: 7.9mm, slimmest OnePlus phone since the OnePlus 6T. Weight: 170g',
  'price': '24,999',
  'prod': ['OnePlus Nord CE 5G']},
 2: {'desc': 'Quad Rear Camera (48 MP + 16 MP + 5 MP + 2 MP) | 16 MP Front Camera',
  'price': '36,999',
  'prod': ['OnePlus 9R 5G']},
 3: {'desc': 'Dual Stereo Speakers for immersive audio experience, The display is protected by Corning Gorilla Glass 5, 64MP AI Quad Camera Setup 8MP Ultrawide-FOV 118 degrees 5MP Macro Sensor 2 MP Depth Sensor Slow Motion Support - Up to 960 fps Portrait | Panaroma | Dual Mode | Vlog Mode | Clone Mode | Time Lapse | Long Exposure | Pro Mode | Night Mode | Short Video Mode | Ultra wide-angle distortion correction | Custom W

In [48]:
def get_random_items(item_count):
  items_set = set()
  while len(items_set) != item_count:
    item_sno = randint(1, len(prod_dict.keys()))
    items_set.add(item_sno)

  items_array = list()
  for item_sno in items_set:
    items_array.append({'sno': item_sno, 'count': randint(1, 3)})
  
  return items_array

def generate_transaction(date, tr_uniq, item_count):
  items = get_random_items(item_count)
  final_list = list()
  for item in items:
    final_list.append([item['sno'], item['count'], date, f'TR-{tr_uniq}'])

  return final_list


def get_dates():
  end_dt = datetime.now()
  duration_obj = datetime.strptime(str(end_dt.year-1), '%Y')
  start_dt = end_dt - timedelta(days=365)
  return start_dt, end_dt

  
def generate_data(start_dt, end_dt):

  count = 1
  transactions = []
  while start_dt < end_dt:
    duration = str(randint(1, 23))
    duration_obj = datetime.strptime(duration, '%H')

    start_dt = start_dt + timedelta(hours=duration_obj.hour, minutes=duration_obj.minute, seconds=duration_obj.second)
    #print(f'Start: {start_dt}')
    
    items = randint(1, 3)
    
    transaction = generate_transaction(start_dt, count, items)
    transactions.extend(transaction)
    count = count + 1
    #print('==================================')
  return transactions


start_dt, end_dt = get_dates()
#print(start_dt)
#print(end_dt)
transactions = generate_data(start_dt, end_dt)

print(len(transactions))
df = pd.DataFrame(transactions, columns=['prod_id', 'qt', 'date', 'tr_id'])

1567


In [49]:
df

,prod_id,qt,date,tr_id
0,2,2,2020-11-12 00:38:10.818415,TR-1
1,7,1,2020-11-12 00:38:10.818415,TR-1
2,9,3,2020-11-12 15:38:10.818415,TR-2
3,1,3,2020-11-12 15:38:10.818415,TR-2
4,8,2,2020-11-12 17:38:10.818415,TR-3
...,...,...,...,...
1562,1,2,2021-11-11 06:38:10.818415,TR-753
1563,5,2,2021-11-11 06:38:10.818415,TR-753
1564,6,1,2021-11-11 06:38:10.818415,TR-753
1565,2,2,2021-11-11 08:38:10.818415,TR-754


In [66]:
def interested_in_analysis(df, prod_id, top_x = 2):
  transactions_with_prod = df[df.prod_id == prod_id].tr_id
  #print(transactions_with_prod.shape)
  #transactions_with_prod.to_csv('2.csv', index = False)
  transactions_with_prod = df[(df.prod_id != prod_id) & df.tr_id.isin(transactions_with_prod)]
  #print(transactions_with_prod.shape)
  #transactions_with_prod.to_csv('3.csv', index = False)
  counts = transactions_with_prod['prod_id'].value_counts(sort=True)
  return counts[:top_x]
#print(df.shape)
#df.to_csv('1.csv', index = False)
prod_id = 1
interested_in = interested_in_analysis(df, prod_id)
print(interested_in)

8    41
9    33
Name: prod_id, dtype: int64


In [73]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
def get_last_date(year, month):
  month_days_dict = { 1: 31, 2: 28, 3: 31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31 }
  last_date = month_days_dict[month]
  if month == 2 and (year % 400 == 0 or year % 4 == 0 and year % 100 != 0):
    last_date = last_date + 1
  return last_date

#def monthly_data(df):
start_dt = df.iloc[0]['date']
end_dt = df.iloc[len(df)-1]['date']

monthly_data = []
while start_dt < end_dt or start_dt.month == end_dt.month:
  last_date = get_last_date(start_dt.year, start_dt.month)
  last_dt = datetime.strptime(f'{start_dt.year}:{start_dt.month}:{last_date} 23:59:59', '%Y:%m:%d %H:%M:%S')
  data = df[(df.date >= start_dt) & (df.date <= last_dt)][['prod_id', 'qt']]
  grp = data.groupby('prod_id', dropna=False)
  y = grp.sum()
  y['prod_id'] = y.index
  print(y)
#  y.to_csv('5.csv')
  monthly_data.append([f'{start_dt.strftime("%B")}-{start_dt.year}', y.to_numpy()])
  start_dt = last_dt + timedelta(days=1)
  start_dt = datetime.strptime(f'{start_dt.year}:{start_dt.month}:{start_dt.day} 00:00:00', '%Y:%m:%d %H:%M:%S')
  

In [ ]:
monthly_data